In [19]:
import psycopg2
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import random
from flask import Flask, jsonify 
from flask_cors import CORS

In [20]:
app = Flask(__name__)
CORS(app)

In [21]:
def connect_db():
    conn = psycopg2.connect(
    # Check these parameters
    host="ep-damp-dust-a11bi2qp-pooler.ap-southeast-1.aws.neon.tech", 
    port="5432",
    database="neondb",
    user="neondb_owner",
    password="Ijov0mkf6Ohl",
    sslmode="require"
    )

    return conn

In [22]:
def assign_unique_jan_1900_dates(df):
    # Generate a list of unique random days in January 1900
    unique_january_days = pd.date_range('1900-01-01', '1900-01-31').to_list()
    import random
    random.shuffle(unique_january_days)
    
    # Get unique dates in the current data
    unique_dates = df['date'].unique()

    # Create a mapping from unique dates to unique random days in January 1900
    date_mapping = {original_date: unique_january_days.pop(0) for original_date in unique_dates}
    
    # Apply the mapping to replace the original dates
    df['date'] = df['date'].map(date_mapping)
    
    return df

In [23]:
def create_drug_sales_time_series(df):
    # Group data by date to get total sales per day
    df_grouped = df.groupby('date').agg({'salesCount': 'sum'}).reset_index()

    # Calculate a rolling average (7-day rolling average)
    df_grouped['Rolling_Avg'] = df_grouped['salesCount'].rolling(window=7).mean()

    # Create the main time series plot
    fig = go.Figure()

    # Add the main sales count line
    fig.add_trace(go.Scatter(
        x=df_grouped['date'],
        y=df_grouped['salesCount'],
        mode='lines+markers',
        name='Daily Sales Count',
        line=dict(width=2),
        marker=dict(size=6)
    ))

    # Add the 7-day rolling average line
    fig.add_trace(go.Scatter(
        x=df_grouped['date'],
        y=df_grouped['Rolling_Avg'],
        mode='lines',
        name='7-day Rolling Average',
        line=dict(width=2, color='red'),
        hoverinfo='skip'
    ))

    # Customize layout and axis labels
    fig.update_layout(
        title='Drug Sales Time Series',
        xaxis_title='Date',
        yaxis_title='Sales Count',
        legend_title='Legend',
        template='plotly_white',  # Clean template with gridlines
        xaxis=dict(
            tickangle=45,
            dtick=86400000.0 * 3,  # Set tick intervals to every 3 days (in milliseconds)
            showgrid=True,
            tickformat="%b %d"  # Format for showing month and day
        ),
        yaxis=dict(
            showgrid=True,
            dtick=10  # Set tick interval for y-axis (adjust based on the scale of your data)
        ),
        hovermode='x unified'  # Better hover display
    )

    plot_json = fig.to_json()
    return  plot_json

In [24]:
@app.route('/1', methods=['GET'])
def fetch_drug_sales_data1():
    try:
        # Connect to the database
        conn = connect_db()
        cursor = conn.cursor()

        # Fetch data from Drug_Sales table
        cursor.execute("""
            SELECT "diseaseId", "date", "regionId", "salesCount"
            FROM "Drug_Sales";
        """)
        
        # Fetch the data and load it into a DataFrame
        drug_sales_data = pd.DataFrame(cursor.fetchall(), columns=['diseaseId', 'date', 'regionId', 'salesCount'])

        # Convert the 'date' column to datetime format
        drug_sales_data['date'] = pd.to_datetime(drug_sales_data['date'])

        # Apply date transformation
        unique_regions = drug_sales_data['regionId'].unique()
        region_dfs = {}
        for region in unique_regions:
            region_df = drug_sales_data[drug_sales_data['regionId'] == region]
            region_dfs[region] = assign_unique_jan_1900_dates(region_df)


        # Generate the Plotly figure
        fig_json = create_drug_sales_time_series(region_dfs[1])
        return jsonify({'plot': fig_json})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

    finally:
        cursor.close()
        conn.close()



In [25]:
@app.route('/2', methods=['GET'])
def fetch_drug_sales_data2():
    try:
        # Connect to the database
        conn = connect_db()
        cursor = conn.cursor()

        # Fetch data from Drug_Sales table
        cursor.execute("""
            SELECT "diseaseId", "date", "regionId", "salesCount"
            FROM "Drug_Sales";
        """)
        
        # Fetch the data and load it into a DataFrame
        drug_sales_data = pd.DataFrame(cursor.fetchall(), columns=['diseaseId', 'date', 'regionId', 'salesCount'])

        # Convert the 'date' column to datetime format
        drug_sales_data['date'] = pd.to_datetime(drug_sales_data['date'])

        # Apply date transformation
        unique_regions = drug_sales_data['regionId'].unique()
        region_dfs = {}
        for region in unique_regions:
            region_df = drug_sales_data[drug_sales_data['regionId'] == region]
            region_dfs[region] = assign_unique_jan_1900_dates(region_df)


        # Generate the Plotly figure
        fig_json = create_drug_sales_time_series(region_dfs[1])
        return jsonify({'plot': fig_json})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

    finally:
        cursor.close()
        conn.close()



In [26]:
@app.route('/3', methods=['GET'])
def fetch_drug_sales_data3():
    try:
        # Connect to the database
        conn = connect_db()
        cursor = conn.cursor()

        # Fetch data from Drug_Sales table
        cursor.execute("""
            SELECT "diseaseId", "date", "regionId", "salesCount"
            FROM "Drug_Sales";
        """)
        
        # Fetch the data and load it into a DataFrame
        drug_sales_data = pd.DataFrame(cursor.fetchall(), columns=['diseaseId', 'date', 'regionId', 'salesCount'])

        # Convert the 'date' column to datetime format
        drug_sales_data['date'] = pd.to_datetime(drug_sales_data['date'])

        # Apply date transformation
        unique_regions = drug_sales_data['regionId'].unique()
        region_dfs = {}
        for region in unique_regions:
            region_df = drug_sales_data[drug_sales_data['regionId'] == region]
            region_dfs[region] = assign_unique_jan_1900_dates(region_df)


        # Generate the Plotly figure
        fig_json = create_drug_sales_time_series(region_dfs[1])
        return jsonify({'plot': fig_json})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

    finally:
        cursor.close()
        conn.close()



In [18]:
if __name__ == '__main__':
    app.run(port=3011)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3011
Press CTRL+C to quit
C:\Users\aagam\AppData\Local\Temp\ipykernel_14728\3841893922.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\aagam\AppData\Local\Temp\ipykernel_14728\3841893922.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\aagam\AppData\Local\Temp\ipykernel_14728\3841893922.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c